In [25]:
import pandas as pd

scores_df=pd.read_csv('../dataframes/scores_df.csv')

track_names_mapping_df=pd.read_csv('../dataframes/track_names_mapping_df.csv')

In [29]:
from mido import MidiFile
import numpy as np
from symusic import TimeUnit,Score
import os

paths='../'+scores_df['scores_paths']

track_names_mapping = {}

for key, value in zip(track_names_mapping_df['track_name_raw'], track_names_mapping_df['track_name_clean']):
    track_names_mapping[key] = value


right_hand_scores_paths_list=[]
left_hand_scores_paths_list=[]

for score_path in paths:

    current_score_pitchs=[]    
    score=Score(score_path)
    track_names=[]

    for track in score.tracks:        
        ## change name of tracks to the clean ones
        track_names.append(track_names_mapping[track.name])
    
    if 'piano' not in track_names:
        right_hand_scores_paths_list.append('')
        left_hand_scores_paths_list.append('')           


    for track in score.tracks:
       
        ## change name of tracks to the clean ones
        track.name=track_names_mapping[track.name]
            
        if track.name == 'piano':            

            track_pitch_avg=[]

            for note in track.notes:              
                track_pitch_avg.append(note.pitch)                    

            current_score_pitchs.append(np.mean(track_pitch_avg))

    if len(current_score_pitchs)>0:   

        ##========================RIGHT HAND SCORE====================##
        ##export right hand
        right_hand_score = Score(ttype=score.ttype)  # Create a new Score object with the same time unit type
        right_hand_score.ticks_per_quarter = score.ticks_per_quarter  # Set the ticks per quarter note

        # Add global events to the new score
        right_hand_score.tempos.extend(score.tempos)
        right_hand_score.time_signatures.extend(score.time_signatures)
        right_hand_score.key_signatures.extend(score.key_signatures)
        right_hand_score.lyrics.extend(score.lyrics)
        right_hand_score.markers.extend(score.markers)


        right_hand_track=current_score_pitchs.index(max(current_score_pitchs))
        # Add the track to the new Score object (tracks should be a list)
        right_hand_score.tracks.append(score.tracks[right_hand_track])

        # Prepare the output path
        output_path = score_path.replace('../Lieder/scores/', '../author-profiling/train data/right hand scores/')
        output_dir = os.path.dirname(output_path)
        os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

        output_file = f"{output_path}_right_hand.mid"
        # Dump each piano new score to a MIDI file
        right_hand_score.dump_midi(output_file)                
        right_hand_scores_paths_list.append(output_file)            

    
        ##===================LEFT HAND SCORE=================================##
        left_hand_score = Score(ttype=score.ttype)  # Create a new Score object with the same time unit type
        left_hand_score.ticks_per_quarter = score.ticks_per_quarter  # Set the ticks per quarter note

        # Add global events to the new score
        left_hand_score.tempos.extend(score.tempos)
        left_hand_score.time_signatures.extend(score.time_signatures)
        left_hand_score.key_signatures.extend(score.key_signatures)
        left_hand_score.lyrics.extend(score.lyrics)
        left_hand_score.markers.extend(score.markers)


        left_hand_track=current_score_pitchs.index(min(current_score_pitchs))
        # Add the track to the new Score object (tracks should be a list)
        left_hand_score.tracks.append(score.tracks[left_hand_track])

        # Prepare the output path
        output_path = score_path.replace('../Lieder/scores/', '../author-profiling/train data/left hand scores/')
        output_dir = os.path.dirname(output_path)
        os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

        # Dump each piano new score to a MIDI file
        output_file = f"{output_path}_left_hand.mid"
        left_hand_score.dump_midi(output_file)                
        left_hand_scores_paths_list.append(output_file)

/home/lafuente/anaconda3/envs/symbolic_music_conda/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/lafuente/anaconda3/envs/symbolic_music_conda/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [34]:
scores_df['left_hand_scores_paths']=left_hand_scores_paths_list
scores_df['right_hand_scores_paths']=right_hand_scores_paths_list

scores_df.to_csv('../dataframes/right_left_hand_scores.csv',index=False)